In [ ]:
import os
import sys
import glob
import pandas as pd
import numpy as np

import spatialdata as sd
from spatialdata_io import xenium
import spatialdata_io
import spatialdata_plot
from spatialdata import SpatialData
from spatialdata import bounding_box_query

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq

from spatialdata import bounding_box_query


saveFolder="/projects/b1146/BharatLab/Yuanqing/Projects/Spatial/IPF"
FigOut="/projects/b1042/Yuanqing/Sptial/data/output"
folderPath="/projects/b1146/BharatLab/20240530__190352__20240530_X1_Bharat"
sampleFile=glob.glob(f'{folderPath}/*__*')
disease=["IPF4","IPF1", "IPF3L", "IPF2", "Donor1", "IPF5", "IPF3R", "Donor2"]
zarr=[saveFolder+f'/Zarr_withCT/{disease[i]}.zarr' for i in range(len(disease))]
pdata=pd.DataFrame({'sam':sampleFile, 'disease':disease, 'Zarr':zarr})
pdata

###https://spatialdata.scverse.org/en/latest/tutorials/notebooks/notebooks/examples/technology_xenium.html

In [ ]:
CTLevel=['AT2','AT1','Ciliated','Club','Basal','KRT17+KRT5-',
           'En',"Capillary.En","Artery.En","Lym.En",
           "Fibro","AlvF","MyoF","SMC","Pericyte","Mesothelial",
           "B","Plasma","CD4T","CD8T","NK",
           "Monocyte","DC","Macro","MoM","IM","Neutrophils","Chemokine.Imm","Prolif.Cells","Mast"]
CTCol=["#E41A1C", "#B43547", "#845172", "#546C9D", "#3983AC", "#3F908E", "#FFB315", "#FFD723", "#FFFA31", "#E9D630",
             "#85658D", "#9B4F9D", "#B75D70", "#D46A43", "#F07816", "#FF9007",
             "#0000FF","#00FFFF","#00FF00","#008B00",
             "#D0A72D", "#B8782A", "#AB5832", "#C1645C",
             "#D87085", "#EE7CAF", "#E685B8", "#CC8BAD", "#B292A3", "#999999"]
cellcol=pd.DataFrame({"CellType": CTLevel,'colSch':CTCol})
cellDic=dict(zip(CTLevel,CTCol))


DicNich={'TLS':[1.0],'MoM':[2.0],'AM':[3.0],'SMC':[4.0],'TLS_GC':[5.0],'Alveolar':[6.0],
         'Airway':[7.0], 'Plasma':[8.0], 'Mixed1':[9.0], 'FF':[10.0], 'Mixed2':[11.0], 'Artery':[12.0]}

NichLab=['TLS','MoM','AM','SMC','TLS_GC','Alveolar','Airway','Plasma','Mixed1','FF','Mixed2','Artery']
NichCol=["#0000FF", "#419486", "#5A9D5A", "#91569A","#E41A1C", "#FFA500", "#FF7F50", "#F6EF32", 
         "#B6742A",'#00FFFF', "#DD87B4", "#999999"]

nichcol=pd.DataFrame({"FNich": NichLab,'NichCol':NichCol})

In [ ]:
IPF2 = sd.read_zarr('/projects/b1146/BharatLab/Yuanqing/Projects/Spatial/IPF/Zarr_withCT/IPF2.zarr')
IPF2.tables['table']

In [ ]:
#######code to remove the 'unknown' cell type
# Filter out cells where 'CellType' is 'unknown'
IPF2_table = IPF2.tables['table']
IPF2_filtered_table = IPF2_table[IPF2_table.obs['CellType'] != 'unknown'].copy()
# Update the spatialdata object
IPF2.tables['table'] = IPF2_filtered_table

if 'cell_id' in IPF2_filtered_table.obs.columns:
    IPF2_filtered_table.obs.set_index('cell_id', inplace=True)

# Filter out shapes to match the filtered table
IPF2_valid_indices = IPF2_filtered_table.obs.index # Keep only valid cell IDs

IPF2.shapes['cell_circles'] = IPF2.shapes['cell_circles'].loc[IPF2_valid_indices]
IPF2.tables['table'].obs.index.name='index'
IPF2.tables['table'].obs['cell_id']=IPF2.tables['table'].obs.index
IPF2.tables['table'].obs

In [ ]:
IPF2.tables['table'].obs['FNich']=np.nan

# Generate new assignments
for i in DicNich.keys():
    ind = pd.Series(IPF2.tables['table'].obs.niches12).isin(DicNich[i])
    IPF2.tables['table'].obs.loc[ind,'FNich'] = i
    
IPF2.tables['table'].obs['FNich']=pd.Categorical(IPF2.tables['table'].obs['FNich'], 
                                                     categories=NichLab, 
                                                     ordered=True)
IPF2.tables['table'].uns["FNich_colors"]=NichCol   
IPF2.tables['table'].obs.head(5)

In [ ]:
IPF2_table.obs.head(3)

In [ ]:
IPF2_table=IPF2.tables['table']
IPF2_B_TLS_table = IPF2_table[IPF2_table.obs['FNich'].isin(['TLS_GC','TLS','FF'])==True].copy()

if 'cell_id' in IPF2_B_TLS_table.obs.columns:
    IPF2_B_TLS_table.obs.set_index('cell_id', inplace=True)

IPF2_valid_indices_B_TLS = IPF2_B_TLS_table.obs.index # Keep only valid cell IDs
IPF2_B_TLS_cellcircle=IPF2.shapes['cell_circles'].loc[IPF2_valid_indices_B_TLS]

IPF2_B_TLS_table.obs.index.name='index'
IPF2_B_TLS_table.obs['cell_id']=IPF2_B_TLS_table.obs.index


In [ ]:
from spatialdata import SpatialData
from spatialdata import bounding_box_query
IPF2_BTLS = SpatialData(
    images=IPF2.images,
    tables={'table': IPF2_B_TLS_table},
    shapes={'cell_circles': IPF2_B_TLS_cellcircle},
    points=IPF2.points
)

In [ ]:
IPF2_BTLS.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    color='CellType',
).pl.show(title='CellType', 
          coordinate_systems="global", 
          figsize=(10, 5))

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

from spatialdata import bounding_box_query

def crop0(x):
    return bounding_box_query(
        x,
        min_coordinate=[15800, 20000],
        max_coordinate=[18000, 22000],
        axes=("x", "y"),
        target_coordinate_system="global",
    )
ZM1=crop0(IPF2_BTLS)

ZM1.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    color='CellType',
    size=0.2, 
    groups=CTLevel,
    palette=CTCol,
).pl.show(title='CellType', 
          coordinate_systems="global", 
          figsize=(10, 10),
         save=f'{FigOut}/IPF2_ZM1.Nich.pdf')

In [ ]:

Zoom1Count=ZM1.tables['table'].obs['CellType'].value_counts()
Zoom1Count

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

ZM1colors = [cellDic[label] for label in Zoom1Count.index]
Zoom1Count.plot.pie(autopct='%1.1f%%', 
                    colors=ZM1colors,  
                    shadow=False, 
                    startangle=0)
plt.tight_layout()
plt.savefig(f"{FigOut}/IPF2_ZM1_Pie.pdf", format="pdf", bbox_inches="tight")


In [ ]:
norm = matplotlib.colors.Normalize(vmin=0, vmax=1.5)
ZM1.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    norm=norm,
    color="CXCL13",
    cmap="Reds",
    method="matplotlib").pl.show(
    coordinate_systems= "global", 
    figsize=(10,4),
    save=f'{FigOut}/IPF2_ZM1_CXCL13.pdf')

In [ ]:
norm2 = matplotlib.colors.Normalize(vmin=0, vmax=1.5)
ZM1.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    norm=norm2,
    color='VWF',
    cmap="Reds",
    method="matplotlib").pl.show(
    coordinate_systems= "global", 
    figsize=(10,10),
    save=f'{FigOut}/IPF2_ZM1_VWF.pdf')

In [ ]:
ZM1.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    norm=norm2,
    color='MKI67',
    cmap="Reds",
    method="matplotlib").pl.show(
    coordinate_systems= "global", 
    figsize=(10,10),
    save=f'{FigOut}/IPF2_ZM1_MKI67.pdf')

In [ ]:
ZM1.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    norm=norm2,
    color='CXCR5',
    cmap="Reds",
    method="matplotlib").pl.show(
    coordinate_systems= "global", 
    figsize=(10,10),
    save=f'{FigOut}/IPF2_ZM1_CXCR5.pdf')

In [ ]:
def obtainCleanDat(disID="IPF4",nichIN=['TLS_GC','TLS','FF']):
    temp = sd.read_zarr(f'/projects/b1146/BharatLab/Yuanqing/Projects/Spatial/IPF/Zarr_withCT/{disID}.zarr')
    temp_table = temp.tables['table']
    temp_filtered_table = temp_table[temp_table.obs['CellType'] != 'unknown'].copy()
    
    temp.tables['table'] = temp_filtered_table
    if 'cell_id' in temp_filtered_table.obs.columns:
        temp_filtered_table.obs.set_index('cell_id', inplace=True)
        
    temp_valid_indices = temp_filtered_table.obs.index # Keep only valid cell IDs
    temp.shapes['cell_circles'] = temp.shapes['cell_circles'].loc[temp_valid_indices]
    temp.tables['table'].obs.index.name='index'
    temp.tables['table'].obs['cell_id']=temp.tables['table'].obs.index
    
    temp.tables['table'].obs['FNich']=np.nan
    for i in DicNich.keys():
        ind = pd.Series(temp.tables['table'].obs.niches12).isin(DicNich[i])
        temp.tables['table'].obs.loc[ind,'FNich'] = i
    temp.tables['table'].obs['FNich']=pd.Categorical(temp.tables['table'].obs['FNich'], 
                                                     categories=NichLab, 
                                                     ordered=True)
    temp.tables['table'].uns["FNich_colors"]=NichCol 
    temp_table=temp.tables['table']
    temp_B_TLS_table = temp_table[temp_table.obs['FNich'].isin(nichIN)==True].copy()
    
    if 'cell_id' in temp_B_TLS_table.obs.columns:
        temp_B_TLS_table.obs.set_index('cell_id', inplace=True)
        
    temp_valid_indices_B_TLS = temp_B_TLS_table.obs.index # Keep only valid cell IDs
    temp_B_TLS_cellcircle=temp.shapes['cell_circles'].loc[temp_valid_indices_B_TLS]
    
    temp_B_TLS_table.obs.index.name='index'
    temp_B_TLS_table.obs['cell_id']=temp_B_TLS_table.obs.index
    
    temp_BTLS = SpatialData(
    images=temp.images,
    tables={'table': temp_B_TLS_table},
    shapes={'cell_circles': temp_B_TLS_cellcircle},
    points=temp.points)
    
    return temp_BTLS



In [ ]:
IPF4_BTLS=obtainCleanDat(disID="IPF4",
                         nichIN=['TLS_GC','TLS','FF'])

In [ ]:
IPF4_BTLS.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    color='CellType',
).pl.show(title='CellType', 
          coordinate_systems="global", 
          figsize=(10, 5))

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
def crop1(x):
    return bounding_box_query(
        x,
        min_coordinate=[31000, 10400],
        max_coordinate=[33600, 12900],
        axes=("x", "y"),
        target_coordinate_system="global")
ZM2=crop1(IPF4_BTLS)

ZM2.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    color='CellType',
    size=0.2, 
    groups=CTLevel,
    palette=CTCol,
).pl.show(title='CellType', 
          coordinate_systems="global", 
          figsize=(10, 10))#,save=f'{FigOut}/IPF4_ZM1.Nich.pdf')

In [ ]:
ZM2.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    norm=norm2,
    color='MKI67',
    cmap="Reds",
    method="matplotlib").pl.show(
    coordinate_systems= "global", 
    figsize=(10,10))

In [ ]:
ZM2.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    norm=norm2,
    color='CXCR5',
    cmap="Reds",
    method="matplotlib").pl.show(
    coordinate_systems= "global", 
    figsize=(10,10))

In [ ]:
ZM2.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    color='CellType',
    size=0.2, 
    groups=CTLevel,
    palette=CTCol,
).pl.show(title='CellType', 
          coordinate_systems="global", 
          figsize=(10, 10),save=f'{FigOut}/IPF4_ZM1.Nich.pdf')

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

Zoom2Count=ZM2.tables['table'].obs['CellType'].value_counts()
ZM2colors = [cellDic[label] for label in Zoom2Count.index]
Zoom2Count.plot.pie(autopct='%1.1f%%', 
                    colors=ZM2colors,  
                    shadow=False, 
                    startangle=0)
plt.tight_layout()
plt.savefig(f"{FigOut}/IPF4_ZM2_Pie.pdf", format="pdf", bbox_inches="tight")


In [ ]:
ZM2.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    norm=norm2,
    color='MKI67',
    cmap="Reds",
    method="matplotlib").pl.show(
    coordinate_systems= "global", 
    figsize=(10,10))

In [ ]:
from spatialdata import SpatialData
from spatialdata import bounding_box_query
IPF5_BTLS=obtainCleanDat(disID="IPF5",
                         nichIN=['TLS_GC','TLS','FF'])

In [ ]:
IPF5_BTLS.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    color='CellType',
).pl.show(title='CellType', 
          coordinate_systems="global", 
          figsize=(10, 5))

In [ ]:
plt.rcParams['pdf.fonttype'] = 42
def crop2(x):
    return bounding_box_query(
        x,
        min_coordinate=[2800, 12500],
        max_coordinate=[8000, 18800],
        axes=("x", "y"),
        target_coordinate_system="global")
ZM3=crop2(IPF5_BTLS)

ZM3.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    color='CellType',
    size=0.2, 
    groups=CTLevel,
    palette=CTCol,
).pl.show(title='CellType', 
          coordinate_systems="global", 
          figsize=(10, 10),
          save=f'{FigOut}/IPF5_ZM3.Nich.pdf')

In [ ]:
ZM3.tables['table'].obs.FNich.value_counts()

In [ ]:
plt.rcParams['pdf.fonttype'] = 42

Zoom3Count=ZM3.tables['table'].obs['CellType'].value_counts()
ZM3colors = [cellDic[label] for label in Zoom3Count.index]
Zoom3Count.plot.pie(autopct='%1.1f%%', 
                    colors=ZM3colors,  
                    shadow=False, 
                    startangle=0)
plt.tight_layout()
plt.savefig(f"{FigOut}/IPF5_ZM3_Pie.pdf", format="pdf", bbox_inches="tight")


In [ ]:
norm2 = matplotlib.colors.Normalize(vmin=0, vmax=1.5)
ZM3.pl.render_images(
    "HE").pl.render_shapes(
    "cell_circles",
    norm=norm2,
    color='CCL19',#MMP9
    cmap="Reds",
    method="matplotlib").pl.show(
    coordinate_systems= "global", 
    figsize=(10,10),
    save=f'{FigOut}/IPF5_ZM3_CCL19.pdf')

